---
We'll use tensorflow to predict the texture in each image: empty, striped or solid.

We should already have `.npy` files in `greyscale-data`.  We'll first load data into various structures for later.  This cell mainly splits the data into training, validation and test folds.

---

In [112]:
import os
import random
import numpy as np


input_directory = 'greyscaled-data'

# Load all the data into an array.  Each element is a tuple: (filename, numpy data).
# The filename structure is "<number>-<color>-<texture>-<shape>-<rotation>.png"
# We'll sort the data first so the later shuffle is consistent.
all_data = [
  (f, np.load(os.path.join(input_directory, f))) for f in os.listdir(input_directory)
]
all_data_sorted = sorted(all_data, key=lambda element: element[0])
random.seed(2)
random.shuffle(all_data_sorted)

# Save 20% of the data for testing (the final, one-shot evaluation of performance).
split_index = int(0.2 * len(all_data_sorted))
test_data = all_data_sorted[0:split_index]
remaining_data = all_data_sorted[split_index:]

# Now save 20% of the remaining data for validation.
split_index = int(0.2 * len(remaining_data))
validation_data = remaining_data[0:split_index]
training_data = remaining_data[split_index:]

# For convenience, get all the pixel data into separate arrays.
training_pixel_data = [pixel_data for _, pixel_data in training_data]
validation_pixel_data = np.array([pixel_data for _, pixel_data in validation_data])
test_pixel_data = np.array([pixel_data for _, pixel_data in test_data])

# Each filename, in its text, has an embedded type of shape.
# As in, "2-red-empty-oval-45.npy"
# We need to convert those classes (the output ground truth) into label arrays.
all_labels = {
  'empty': [1., 0., 0.],
  'striped': [0., 1., 0.],
  'solid': [0., 0., 1.],
}
training_labels = [
  all_labels[filename.split('-')[2]] for filename, _ in training_data
]
validation_labels = [
  all_labels[filename.split('-')[2]] for filename, _ in validation_data
]
test_labels = [
  all_labels[filename.split('-')[2]] for filename, _ in test_data
]

---
setup tensorflow

---

In [156]:
import random
import tensorflow as tf


learning_rate = 0.00005
regularization_factor = 1e-4
card_width, card_height = 150, 150
first_hidden_layer_size, second_hidden_layer_size, third_hidden_layer_size = 512, 512, 512

graph = tf.Graph()
with graph.as_default():
  # Setup the training steps.
  tf_training_data = tf.placeholder(tf.float32, shape=[None, card_width*card_height])
  tf_training_labels = tf.placeholder(tf.float32, shape=[None, 3])
  
  # Create hidden layers of ReLUs.
  first_hidden_weights = tf.Variable(
    tf.truncated_normal([card_width*card_height, first_hidden_layer_size]), name='first_hidden_weights')
  first_hidden_biases = tf.Variable(
    tf.zeros([first_hidden_layer_size]), name='first_hidden_biases')
  first_hidden_layer = tf.nn.relu(tf.matmul(tf_training_data, first_hidden_weights) + first_hidden_biases)
  second_hidden_weights = tf.Variable(
    tf.truncated_normal([first_hidden_layer_size, second_hidden_layer_size]), name='second_hidden_weights')
  second_hidden_biases = tf.Variable(
    tf.zeros([second_hidden_layer_size]), name='second_hidden_biases')
  second_hidden_layer = tf.nn.relu(tf.matmul(first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  third_hidden_weights = tf.Variable(
    tf.truncated_normal([second_hidden_layer_size, third_hidden_layer_size]), name='third_hidden_weights')
  third_hidden_biases = tf.Variable(
    tf.zeros([third_hidden_layer_size]), name='third_hidden_biases')
  third_hidden_layer = tf.nn.relu(tf.matmul(second_hidden_layer, third_hidden_weights) + third_hidden_biases)
  
  # Build the output layer.
  output_weights = tf.Variable(tf.truncated_normal([third_hidden_layer_size, 3]), name='output_weights')
  output_biases = tf.Variable(tf.zeros([3]), name='output_biases')
  output_logits = tf.matmul(third_hidden_layer, output_weights) + output_biases
  training_estimate = tf.nn.softmax(output_logits)

  # Calculate loss and setup the optimizer.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_logits, tf_training_labels))
  l2_regularization = (tf.nn.l2_loss(output_weights) +
                       tf.nn.l2_loss(first_hidden_weights) +
                       tf.nn.l2_loss(second_hidden_weights) +
                       tf.nn.l2_loss(third_hidden_weights))
  loss += regularization_factor * l2_regularization
  training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

  # Setup validation.  We have to reshape into a "dense tensor"
  # by, essentially, combining this array of arrays into a true matrix.
  tf_validation_pixel_data = tf.constant(
    validation_pixel_data.reshape((-1, card_width*card_height)).astype(np.float32))
  validation_first_hidden_layer = tf.nn.relu(
    tf.matmul(tf_validation_pixel_data, first_hidden_weights) + first_hidden_biases)
  validation_second_hidden_layer = tf.nn.relu(
    tf.matmul(validation_first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  validation_third_hidden_layer = tf.nn.relu(
    tf.matmul(validation_second_hidden_layer, third_hidden_weights) + third_hidden_biases)
  validation_logits = tf.matmul(validation_third_hidden_layer, output_weights) + output_biases
  validation_estimate = tf.nn.softmax(validation_logits)

  # Setup the final test run.
  tf_test_pixel_data = tf.constant(
    test_pixel_data.reshape((-1, card_width*card_height)).astype(np.float32))
  test_first_hidden_layer = tf.nn.relu(
    tf.matmul(tf_test_pixel_data, first_hidden_weights) + first_hidden_biases)
  test_second_hidden_layer = tf.nn.relu(
    tf.matmul(test_first_hidden_layer, second_hidden_weights) + second_hidden_biases)
  test_third_hidden_layer = tf.nn.relu(
    tf.matmul(test_second_hidden_layer, third_hidden_weights) + third_hidden_biases)
  test_logits = tf.matmul(test_third_hidden_layer, output_weights) + output_biases
  test_estimate = tf.nn.softmax(test_logits)

---
aside: create a small function to calculate the accuracy of a set of predictions
  
---

In [157]:
def calculate_accuracy(predictions, ground_truth):
  """Determine what proportion of predictions are accurate based on ground truth."""
  correctness = np.sum(np.argmax(predictions, 1) == np.argmax(ground_truth, 1))
  return 100. * correctness / predictions.shape[0]

---
and run the optimizer in batches

---

In [ ]:
accuracies = []
total_iterations = 300
batch_size = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  variable_saver = tf.train.Saver()

  for iteration in range(total_iterations + 1):
    batch_data = random.sample(training_data, batch_size)
    batch_input_pixel_data = [pixel_data for _, pixel_data in batch_data]
    batch_labels = [
      all_labels[filename.split('-')[2]] for filename, _ in batch_data
    ]
    batch_training_data = {
      tf_training_data: batch_input_pixel_data,
      tf_training_labels: batch_labels,
    }
    _, step_loss, training_predictions = session.run(
      [training_step, loss, training_estimate], feed_dict=batch_training_data)
  
    if (iteration % (total_iterations / 20)) == 0:
      training_accuracy = calculate_accuracy(training_predictions, batch_labels)
      validation_accuracy = calculate_accuracy(validation_estimate.eval(), validation_labels)
      accuracies.append((iteration, training_accuracy, validation_accuracy))
      if (iteration % (total_iterations / 10)) == 0:
        print 'iteration: %s -> loss: %s, training accuracy: %0.1f%%, validation accuracy: %0.1f%%' % (
          iteration, step_loss, training_accuracy, validation_accuracy)
  variable_saver.save(session, '/tmp/detect-texture-model.ckpt', latest_filename='detect-texture-checkpoint-list')
  print '\ntest accuracy: %0.1f%%' % calculate_accuracy(test_estimate.eval(), test_labels)

iteration: 0 -> loss: 235345.0, training accuracy: 31.0%, validation accuracy: 32.4%
iteration: 30 -> loss: 2503.9, training accuracy: 74.0%, validation accuracy: 65.3%
iteration: 60 -> loss: 2224.53, training accuracy: 77.0%, validation accuracy: 68.7%
iteration: 90 -> loss: 2718.89, training accuracy: 65.0%, validation accuracy: 64.5%
iteration: 120 -> loss: 807.605, training accuracy: 91.0%, validation accuracy: 73.7%
iteration: 150 -> loss: 691.211, training accuracy: 94.0%, validation accuracy: 73.7%
iteration: 180 -> loss: 929.777, training accuracy: 86.0%, validation accuracy: 71.8%
iteration: 210 -> loss: 1787.45, training accuracy: 72.0%, validation accuracy: 73.7%


---
plot the accuracy vs iteration number

---

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt


iterations, training_accuracies, validation_accuracies = zip(*accuracies)
plt.plot(iterations, training_accuracies, 'r-', label='training')
plt.plot(iterations, validation_accuracies, 'g-', label='validation')
axes = plt.gca()
_ = axes.set_ylim([0, 110])
_ = plt.xlabel('iteration')
_ = plt.ylabel('accuracy')
_ = plt.legend(loc='best')